In [4]:
import tkinter as tk
from tkinter import messagebox
import random
import os

# List of words with hints for the game
words_with_hints = [
    ("python", "A popular programming language."),
    ("hangman", "A classic word-guessing game."),
    ("programming", "The act of writing computer programs."),
    ("coding", "Another word for programming."),
    ("challenge", "A task or problem that tests a person's abilities."),
    ("project", "An individual or collaborative enterprise."),
    ("java", "A high-level programming language."),
    ("out", "A direction or movement away from the inside."),
    ("jackpot", "A large cash prize in a game or lottery."),
    ("count", "To determine the total number of something."),
    ("country", "A nation with its own government."),
    ("school", "An institution for educating children."),
    ("class", "A group of students meeting regularly."),
    ("fun", "Enjoyment, amusement, or lighthearted pleasure."),
    ("elephant", "A large mammal with a trunk."),
    ("guitar", "A musical instrument with strings."),
    ("ocean", "A vast body of saltwater."),
    ("bicycle", "A two-wheeled vehicle powered by pedaling."),
    ("computer", "A device for processing data.")
]

# Initialize variables
word_to_guess, hint = random.choice(words_with_hints)
guessed_letters = []
attempts = 6
attempts_used = 0

# Log file path
log_file = "hangman_log.txt"

# Function to save the game state to a log file with headings
def save_game(result=""):
    try:
        file_exists = os.path.exists(log_file)
        with open(log_file, "a") as file:
            if not file_exists:
                file.write("Word=Word to Guess|Attempts Left|Attempts Used|Result\n")
            if result:
                file.write(f"Word={word_to_guess}\n")
                file.write(f"Attempts Used={attempts_used}\n")
                file.write(f"Result={result}\n")
    except IOError as e:
        messagebox.showerror("Error", f"Failed to save game: {e}")

# Function to load the last game state from the log file
def load_game():
    global word_to_guess, guessed_letters, attempts, attempts_used, hint
    try:
        if os.path.exists(log_file):
            with open(log_file, "r") as file:
                lines = file.readlines()
                if len(lines) > 1:  # Skip the heading line
                    last_line = lines[-1].strip()
                    if last_line:
                        parts = last_line.split('|')
                        if len(parts) == 4:
                            word_part, attempts_part, attempts_used_part, result_part = parts
                            if 'Word=' in word_part and 'Attempts Left=' in attempts_part and 'Attempts Used=' in attempts_used_part:
                                word_to_guess = word_part.split('=')[1]
                                attempts = int(attempts_part.split('=')[1])
                                attempts_used = int(attempts_used_part.split('=')[1])
                                guessed_letters.clear()
                                hint = next(hint for word, hint in words_with_hints if word == word_to_guess)
                                update_word_display()
                                update_attempts_display()
                                draw_hangman()
    except IOError as e:
        messagebox.showerror("Error", f"Failed to load game: {e}")

# Create a window
window = tk.Tk()
window.title("Hangman Game")
window.configure(bg="light blue")

# Function to check if the game is over
def is_game_over():
    return check_win() or check_loss()

# Function to check if the player has won
def check_win():
    return all(letter in guessed_letters for letter in word_to_guess)

# Function to check if the player has lost
def check_loss():
    return attempts == 0

# Function to handle a letter guess
def guess_letter():
    global attempts, attempts_used
    letter = letter_entry.get().lower()
    if letter in guessed_letters:
        messagebox.showinfo("Hangman", f"You've already guessed '{letter}'")
    elif letter in word_to_guess:
        guessed_letters.append(letter)
        update_word_display()
        if check_win():
            messagebox.showinfo("Hangman", "Congratulations! You won!")
            save_game(result="Win")
            reset_game()
    else:
        guessed_letters.append(letter)
        attempts -= 1
        attempts_used += 1
        update_attempts_display()
        draw_hangman()
        if check_loss():
            messagebox.showinfo("Hangman", f"Game Over! The word was '{word_to_guess}'")
            save_game(result="Lose")
            reset_game()
    letter_entry.delete(0, tk.END)
    save_game()

# Function to reset the game
def reset_game():
    global word_to_guess, guessed_letters, attempts, attempts_used, hint
    word_to_guess, hint = random.choice(words_with_hints)
    guessed_letters = []
    attempts = 6
    attempts_used = 0
    update_word_display()
    update_attempts_display()
    draw_hangman()

# Function to update the word display
def update_word_display():
    display_word = ""
    for letter in word_to_guess:
        if letter in guessed_letters:
            display_word += letter
        else:
            display_word += "_"
        display_word += " "
    word_label.config(text=display_word.strip())
    hint_label.config(text=f"Hint: {hint}")

# Function to update the attempts display
def update_attempts_display():
    attempts_label.config(text=f"Attempts left: {attempts}")

# Function to draw the hangman figure
def draw_hangman():
    canvas.delete("hangman")
    if attempts < 6:
        canvas.create_oval(125, 50, 175, 100, width=4, tags="hangman")  # Head
    if attempts < 5:
        canvas.create_line(150, 100, 150, 200, width=4, tags="hangman")  # Body
    if attempts < 4:
        canvas.create_line(150, 120, 100, 170, width=4, tags="hangman")  # Left Arm
    if attempts < 3:
        canvas.create_line(150, 120, 200, 170, width=4, tags="hangman")  # Right Arm
    if attempts < 2:
        canvas.create_line(150, 200, 100, 250, width=4, tags="hangman")  # Left Leg
    if attempts < 1:
        canvas.create_line(150, 200, 200, 250, width=4, tags="hangman")  # Right Leg

# Create GUI elements
word_label = tk.Label(window, text="", font=("Arial", 24), bg="light blue")
hint_label = tk.Label(window, text="", font=("Arial", 16), bg="light blue")
attempts_label = tk.Label(window, text="", font=("Arial", 16), bg="light blue")
attempts_used_label = tk.Label(window, text="", font=("Arial", 16), bg="light blue")
letter_entry = tk.Entry(window, width=5, font=("Arial", 16))
guess_button = tk.Button(window, text="Guess", command=guess_letter)
reset_button = tk.Button(window, text="Reset", command=reset_game)
canvas = tk.Canvas(window, width=300, height=300, bg="light blue")
canvas.create_line(50, 250, 250, 250, width=4)  # Base Line
canvas.create_line(200, 250, 200, 50, width=4)  # Post
canvas.create_line(100, 50, 200, 50, width=4)   # Beam
canvas.create_line(150, 50, 150, 75, width=4)   # Rope
canvas.pack()

# Pack GUI elements
word_label.pack()
hint_label.pack()
attempts_label.pack()
attempts_used_label.pack()
letter_entry.pack()
guess_button.pack()
reset_button.pack()

# Load the last game state if available
try:
    load_game()
except Exception as e:
    messagebox.showerror("Error", f"Failed to load game state: {e}")

# Update initial displays
try:
    update_word_display()
    update_attempts_display()
    draw_hangman()
except Exception as e:
    messagebox.showerror("Error", f"Failed to update displays: {e}")

# Run the application
try:
    window.mainloop()
except Exception as e:
    messagebox.showerror("Error", f"Failed to run application: {e}")